In [2]:
import pandas as pd
from sklearn.metrics import recall_score,  precision_score

In [3]:
smote = pd.read_csv('smote_xtest_output.csv')

In [4]:
rus = pd.read_csv('rus_xtest_output.csv')

In [6]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

smote['probability_bin'] = pd.qcut(smote['predicted'], 10, duplicates='drop')
x = smote.groupby('probability_bin').agg(dict(true = ['mean', 'sum', 'count']))
x.columns = ['fraud_proportion_bin', 'fraud_count_bin', 'frequency']

thre = [t.right for t in x.index]
recall = [recall_score(smote['true'], smote['predicted'] >= t) for t in thre]
precision = [precision_score(smote['true'], smote['predicted'] >= t) for t in thre]
fpr = [1 - recall_score(smote['true'], smote['predicted'] >= t, pos_label=0) for t in thre]


x['Recall'] = recall
x['Precision'] = precision
x['FPR'] = fpr

x

,fraud_proportion_bin,fraud_count_bin,frequency,recall,precision,fpr
probability_bin,,,,,,
"(-0.001, 0.115]",0.005080,48,9449,0.985401,0.038131,0.896172
"(0.115, 0.149]",0.004863,46,9459,0.971411,0.042266,0.793594
"(0.149, 0.175]",0.006780,64,9439,0.951946,0.047295,0.691356
"(0.175, 0.222]",0.008467,80,9448,0.926703,0.053803,0.587572
"(0.222, 0.277]",0.017251,163,9449,0.878650,0.061078,0.486968
"(0.277, 0.334]",0.018695,177,9468,0.824209,0.071625,0.385158
"(0.334, 0.384]",0.023863,225,9429,0.755170,0.087677,0.283304
"(0.384, 0.454]",0.036092,341,9448,0.652372,0.113228,0.184202
"(0.454, 0.612]",0.051011,482,9449,0.505474,0.175817,0.085429


In [8]:
dict(true = ['mean', 'sum', 'count'])

{'true': ['mean', 'sum', 'count']}

In [7]:
rus['probability_bin'] = pd.qcut(rus['predicted'], 10, duplicates='drop')
y = rus.groupby('probability_bin').agg(dict(true = ['mean', 'sum', 'count']))
y.columns = ['fraud_proportion_bin', 'fraud_count_bin', 'frequency']

thre = [t.right for t in x.index]
recall = [recall_score(rus['true'], rus['predicted'] >= t) for t in thre]
precision = [precision_score(rus['true'], rus['predicted'] >= t) for t in thre]
fpr = [1 - recall_score(smote['true'], rus['predicted'] >= t, pos_label=0) for t in thre]

y['Recall'] = recall
y['Precision'] = precision
y['FPR'] = fpr

y

,fraud_proportion_bin,fraud_count_bin,frequency,recall,precision,fpr
probability_bin,,,,,,
"(-0.000732, 0.18]",0.004229,40,9458,0.999696,0.034999,0.993750
"(0.18, 0.192]",0.005720,54,9441,0.999392,0.035100,0.990482
"(0.192, 0.225]",0.007939,75,9447,0.989964,0.037644,0.912422
"(0.225, 0.332]",0.010371,98,9449,0.951034,0.046984,0.695479
"(0.332, 0.391]",0.014605,138,9449,0.932178,0.049744,0.642014
"(0.391, 0.43]",0.023812,225,9449,0.917883,0.053618,0.584096
"(0.43, 0.451]",0.026884,254,9448,0.882908,0.060290,0.496146
"(0.451, 0.522]",0.034716,328,9448,0.727494,0.086391,0.277371
"(0.522, 0.635]",0.060853,575,9449,0.498175,0.146603,0.104552


In [10]:
import numpy as np

In [11]:
# Step 1: Sort DataFrame based on fraud_proportion_bin
x = x.sort_values('fraud_proportion_bin')

# Step 2: Assign tier numbers
x['Tier'] = range(1, len(x) + 1)

# Step 3: Determine tier thresholds and criteria
# Modify the conditions below based on your requirements
tier_thresholds = {
    'Tier 1': {'fraud_count_bin': 0, 'precision': 0.95},
    'Tier 2': {'fraud_count_bin': 10, 'precision': 0.90},
    'Tier 3': {'fraud_count_bin': 20, 'precision': 0.80},
    'Tier 4': {'fraud_count_bin': 30, 'precision': 0.70},
    # Add more tiers and their thresholds as needed
}

# Step 4: Assign tier labels to the original data
smote['Tier'] = pd.cut(smote['predicted'], bins=[-np.inf] + list(x.index) + [np.inf], labels=x['Tier'])

# Assign tier labels based on thresholds and criteria
for tier, thresholds in tier_thresholds.items():
    condition = (smote['fraud_count_bin'] >= thresholds['fraud_count_bin']) & (smote['Precision'] >= thresholds['precision'])
    smote.loc[condition, 'Tier'] = tier


TypeError: float() argument must be a string or a real number, not 'pandas._libs.interval.Interval'